In [ ]:
# 必要なライブラリのインストール
!pip3 install janome # 日本語形態素解析

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from janome.tokenizer import Tokenizer
import pandas as pd
from IPython.display import display
from sklearn.metrics.pairwise import cosine_similarity

# 日本語テキストのトークン化を行うTokenizerの設定
tokenizer = Tokenizer()

def tokenize(text):
    """テキストをトークン化し、意味を持つ単語（名詞、動詞、形容詞）のみを抽出してリストとして返します。"""
    tokens = tokenizer.tokenize(text)
    return [token.surface for token in tokens if token.part_of_speech.split(',')[0] in ['名詞', '動詞', '形容詞']]

# サンプルデータセットの定義
documents = [
    "ディープラーニングは人工知能の一機能です",
    "人工知能と機械学習はコンピュータサイエンスの一部です",
    "ディープラーニングはほとんどのAIアプリケーションで使用されています"
]


# TfidfVectorizerの初期化とパラメータ説明
vectorizer = TfidfVectorizer(
    tokenizer=tokenize,    # 文書をトークン化するための関数: 上で関数として定義
    token_pattern=None     # 上のとおり、カスタムトークナイザを使用するのでtoken_patternは無効とする（このオプションを指定しないとwarningがでるので指定する)
)

# 文書のTF-IDFベクトルを計算
tfidf_matrix = vectorizer.fit_transform(documents)

# クエリの定義とTF-IDFベクトルの計算
query = "AIとディープラーニング"
query_tfidf = vectorizer.transform([query])

# 文書間のコサイン類似度の計算
cosine_similarities = cosine_similarity(query_tfidf, tfidf_matrix).flatten()

#===============================================
# 出力
for i, doc in enumerate(documents):
    tokens = tokenize(doc)
    print(f"対象文書{i}: {doc}")
    print(f"\t分かち書きの結果（トークン化） => {tokens}\n")

print(f"\nクエリ文書: {query}")
query_tokens = tokenize(query)
print(f"\t分かち書きの結果（トークン化） => {query_tokens}\n")

print("==============================================================")
# 文書とクエリのTF-IDFベクトルをテーブル形式で表示
feature_names = vectorizer.get_feature_names_out()
print("\n対象文章のTF-IDF行列:")
df_tfidf = pd.DataFrame(tfidf_matrix.toarray(), columns=feature_names)
display(df_tfidf)

print("クエリのTF-IDF行列:")
query_vector_df = pd.DataFrame(query_tfidf.toarray(), columns=feature_names)
display(query_vector_df)

# 類似度結果の表示
print("クエリ文書との類似文書とその類似度:")
similarity_df = pd.DataFrame({
    'Document': documents,
    'Similarity': cosine_similarities
}).sort_values(by='Similarity', ascending=False)
display(similarity_df)